In [1]:
! curl http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx -o retail.xlsx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22.6M  100 22.6M    0     0  17.0M      0  0:00:01  0:00:01 --:--:-- 17.0M


In [2]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori

In [3]:
df = pd.read_excel("retail.xlsx")
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [4]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis = 0, subset=['InvoiceNo'], inplace = True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
df.groupby('Country').count().reset_index().sort_values(
  'InvoiceNo', ascending = False).head()

,Country,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
36,United Kingdom,487622,487622,486167,487622,487622,487622,354345
14,Germany,9042,9042,9042,9042,9042,9042,9042
13,France,8408,8408,8408,8408,8408,8408,8342
10,EIRE,7894,7894,7894,7894,7894,7894,7238
31,Spain,2485,2485,2485,2485,2485,2485,2485


In [5]:
print(df.groupby('Country').count().reset_index().sort_values(
  'InvoiceNo', ascending = False)["InvoiceNo"].sum())

532621


In [6]:
#フランスのデータを使う
basket_fr = (df[df['Country']=="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket_fr.head(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537468,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
def sum_to_boolean(x):
  if x<=0:
    return 0
  else:
    return 1
basket_fr_final = basket_fr.applymap(sum_to_boolean)
basket_fr_final.head(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537468,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537897,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
frequent_itemsets_fr = apriori(basket_fr_final, min_support = 0.06,
  use_colnames = True)
frequent_itemsets_fr.sort_values('support', ascending = False)

,support,itemsets
34,0.765306,(POSTAGE)
35,0.188776,(RABBIT NIGHT LIGHT)
41,0.181122,(RED TOADSTOOL LED NIGHT LIGHT)
32,0.170918,(PLASTERS IN TIN WOODLAND ANIMALS)
29,0.168367,(PLASTERS IN TIN CIRCUS PARADE)
...,...,...
119,0.061224,"(LUNCH BAG APPLE DESIGN, POSTAGE, LUNCH BAG RE..."
33,0.061224,(POPPY'S PLAYHOUSE KITCHEN)
43,0.061224,(RETROSPOT PARTY BAG + STICKER SET)
83,0.061224,"(PACK OF 6 SKULL PAPER CUPS, POSTAGE)"


In [19]:
a_rules = association_rules(frequent_itemsets_fr, metric = "confidence",
  min_threshold = 0.5)
a_rules = a_rules.sort_values('lift',ascending = False)
a_rules[a_rules["lift"] >= 5.0]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
11,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
73,(ALARM CLOCK BAKELIKE RED),"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",0.094388,0.073980,0.063776,0.675676,9.133271,0.056793,2.855230
70,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE RED),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689
81,"(ALARM CLOCK BAKELIKE GREEN, POSTAGE)",(ALARM CLOCK BAKELIKE RED),0.084184,0.094388,0.071429,0.848485,8.989353,0.063483,5.977041
...,...,...,...,...,...,...,...,...,...
64,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER CUPS),0.132653,0.137755,0.102041,0.769231,5.584046,0.083767,3.736395
112,(SET/6 RED SPOTTY PAPER CUPS),"(SET/20 RED RETROSPOT PAPER NAPKINS, POSTAGE)",0.137755,0.109694,0.084184,0.611111,5.571059,0.069073,2.289359
111,"(SET/20 RED RETROSPOT PAPER NAPKINS, POSTAGE)",(SET/6 RED SPOTTY PAPER CUPS),0.109694,0.137755,0.084184,0.767442,5.571059,0.069073,3.707653
113,(SET/20 RED RETROSPOT PAPER NAPKINS),"(SET/6 RED SPOTTY PAPER CUPS, POSTAGE)",0.132653,0.117347,0.084184,0.634615,5.408027,0.068617,2.415682


In [10]:
print( basket_fr_final['CHILDRENS CUTLERY SPACEBOY'].sum())
print( basket_fr_final['CHILDRENS CUTLERY DOLLY GIRL'].sum())

27
28


In [11]:
basket_fr_final['SET/20 RED RETROSPOT PAPER NAPKINS'].sum()

52

In [12]:
basket_uk = (df[df['Country']=="United Kingdom"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket_final_uk = basket_uk.applymap(sum_to_boolean)


In [13]:
frequent_itemsets_uk = apriori(basket_final_uk, min_support = 0.01,
  use_colnames = True)
a_rules_uk = association_rules(frequent_itemsets_uk, metric = "confidence",
  min_threshold = 0.6)
a_rules_uk.sort_values('lift',ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
327,"(HERB MARKER ROSEMARY, HERB MARKER PARSLEY)",(HERB MARKER THYME),0.011089,0.012321,0.010553,0.951691,77.240055,0.010417,20.444951
330,(HERB MARKER THYME),"(HERB MARKER ROSEMARY, HERB MARKER PARSLEY)",0.012321,0.011089,0.010553,0.856522,77.240055,0.010417,6.892409
323,(HERB MARKER ROSEMARY),"(HERB MARKER MINT, HERB MARKER THYME)",0.012375,0.010714,0.010232,0.826840,77.173095,0.010099,5.713126
322,"(HERB MARKER MINT, HERB MARKER THYME)",(HERB MARKER ROSEMARY),0.010714,0.012375,0.010232,0.955000,77.173095,0.010099,21.947227
314,"(HERB MARKER MINT, HERB MARKER THYME)",(HERB MARKER PARSLEY),0.010714,0.012214,0.010071,0.940000,76.960439,0.009940,16.463099


In [14]:
a_rules_uk = association_rules(frequent_itemsets_uk, metric = "confidence",
  min_threshold = 0.6)
a_rules_uk.sort_values('lift',ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
327,"(HERB MARKER ROSEMARY, HERB MARKER PARSLEY)",(HERB MARKER THYME),0.011089,0.012321,0.010553,0.951691,77.240055,0.010417,20.444951
330,(HERB MARKER THYME),"(HERB MARKER ROSEMARY, HERB MARKER PARSLEY)",0.012321,0.011089,0.010553,0.856522,77.240055,0.010417,6.892409
323,(HERB MARKER ROSEMARY),"(HERB MARKER MINT, HERB MARKER THYME)",0.012375,0.010714,0.010232,0.826840,77.173095,0.010099,5.713126
322,"(HERB MARKER MINT, HERB MARKER THYME)",(HERB MARKER ROSEMARY),0.010714,0.012375,0.010232,0.955000,77.173095,0.010099,21.947227
314,"(HERB MARKER MINT, HERB MARKER THYME)",(HERB MARKER PARSLEY),0.010714,0.012214,0.010071,0.940000,76.960439,0.009940,16.463099
...,...,...,...,...,...,...,...,...,...
61,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.048749,0.103820,0.030535,0.626374,6.033290,0.025474,2.398601
72,(JUMBO STORAGE BAG SUKI),(JUMBO BAG RED RETROSPOT),0.060535,0.103820,0.037392,0.617699,5.949737,0.031108,2.344176
346,"(JUMBO BAG ALPHABET, JUMBO BAG APPLES)",(JUMBO BAG RED RETROSPOT),0.019660,0.103820,0.012107,0.615804,5.931481,0.010066,2.332611
65,(JUMBO BAG DOLLY GIRL DESIGN),(JUMBO BAG RED RETROSPOT),0.022714,0.103820,0.013982,0.615566,5.929191,0.011624,2.331169
